<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/master/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
baseURL = 'https://www.soccerbase.com'

In [ ]:
request = requests.get("https://www.soccerbase.com/players/home.sd")
soup = BeautifulSoup(request.content, "lxml")
testing = soup.find('div', id = 'players-menu')#attrs = {'class': 'soccerColumn soccerColumnLast'})
for div in testing.find_all('div', class_ = 'soccerColumn'):
  if (re.search('Competition', div.text) != None):
    comp = div
threads = []
compQ = queue.Queue()
for num, competition in enumerate(comp.find_all('a'), 1):
  t = threading.Thread(target = readCompetitionPage, args = (competition.text, baseURL + competition['href'], compQ))
  threads.append(t)
  t.start()
  #print(num, baseURL + competition['href'])
  #print(competition.text)
for t in threads:
  t.join()
counter = 1
while not compQ.empty():
  print(counter, compQ.get()[0])
  counter += 1
#for li in testing.find_all('ol'):
#  print(li)

1 /tournaments/tournament.sd?comp_id=1
2 /tournaments/tournament.sd?comp_id=2
3 /tournaments/tournament.sd?comp_id=3
4 /tournaments/tournament.sd?comp_id=4
5 /tournaments/tournament.sd?comp_id=12
6 /tournaments/tournament.sd?comp_id=13
7 /tournaments/tournament.sd?comp_id=14
8 /tournaments/tournament.sd?comp_id=15
9 /tournaments/tournament.sd?comp_id=66
10 /tournaments/tournament.sd?comp_id=64
11 /tournaments/tournament.sd?comp_id=19
12 /tournaments/tournament.sd?comp_id=20
13 /tournaments/tournament.sd?comp_id=21
14 /tournaments/tournament.sd?comp_id=22
15 /tournaments/tournament.sd?comp_id=116
16 /tournaments/tournament.sd?comp_id=25
17 /tournaments/tournament.sd?comp_id=112
18 /tournaments/tournament.sd?comp_id=23
19 /tournaments/tournament.sd?comp_id=114
20 /tournaments/tournament.sd?comp_id=76
21 /tournaments/tournament.sd?comp_id=206
22 /tournaments/tournament.sd?comp_id=124
23 /tournaments/tournament.sd?comp_id=24
24 /tournaments/tournament.sd?comp_id=111
25 /tournaments/tournam

In [ ]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
  return timeIt

In [ ]:
#From player page for a team
def getTeamData(team, url, q):
  names = [team]
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
  for elem in test.find_all('b'):
    name = elem.find('a')
    names.append(name.text)
  #return names
  q.put(names)

#From competition page of teams
## seasons: dataframe consisted of each season value with the season date

In [ ]:
import threading
import queue
@timer
def readCompetitionPage(compURL):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  seasons = tournament_season_selector(soup)
  print(seasons)
  teamNames = soup.find('table', class_ = 'table')
  teams = {}
  threads = []
  q = queue.Queue()

  for num, url in enumerate(teamNames.find_all('a'), 1):
    #print(num, url['href'])
    t = threading.Thread(target = getTeamData, args = (url.text, baseURL + url['href'], q))
    threads.append(t)
    t.start()
    #print(url.text)
    #getTeamData(url.text, baseURL+url['href'], q)
    #print(getTeamData(baseURL + url['href']))
  #print(num)
  for t in threads:
    t.join()
  while not q.empty():
    teamData = q.get()
    teams[teamData[0]] = teamData[1:]

  print(teams)
  '''
  for team, players in teams.items():
    print("TEAM:", team, end = '\nPLAYERS: ')
    print(*players, sep=', ')
    print()
'''

In [ ]:
readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')

   value     date
0   1792  2020/21
1   1725  2019/20
2   1668  2018/19
3   1597  2017/18
4   1511  2016/17
5   1462  2015/16
6   1395  2014/15
7   1316  2013/14
8   1172  2012/13
9   1007  2011/12
10    45  2010/11
11    46  2009/10
12   321  2008/09
13   322  2007/08
14   323  2006/07
15   324  2005/06
16   325  2004/05
17   326  2003/04
18   327  2002/03
19   328  2001/02
20   329  2000/01
21   330  1999/00
22   331  1998/99
23   332  1997/98
24   333  1996/97
25   334  1995/96
26   335  1994/95
27   336  1993/94
28   337  1992/93
29   338  1991/92
30   339  1990/91
31   340  1989/90
32   341  1988/89
33   342  1987/88
34   343  1986/87
35   344  1985/86
36   345  1984/85
37   346  1983/84
38   347  1982/83
39   348  1981/82
40   349  1980/81
41   350  1979/80
42   351  1978/79
43   352  1977/78
44   353  1976/77
45   354  1975/76
46   355  1974/75
47   356  1973/74
48   357  1972/73
49   358  1971/72
50   359  1970/71
51   360  1969/70
52   361  1968/69
53   362  1967/68
54   363  

In [ ]:
import re
import pandas as pd
def tournament_season_selector(soup):
  df = pd.DataFrame(columns = ["value", "date"])
  list = soup.find("select", id = "seasonSelect")
  for data in list.find_all("option")[1:]:
    df = df.append({"value": data["value"], "date" : data.text}, ignore_index = True)
  return df
